# Sentinel-2 File Unzip And Management

This program takes the downloaded Sentinel-2 imagery and unzips the files programmatically based on the file naming conventions.

In [1]:
import os
import shutil

## Unzipping files

The first section of this script deals with unzipping the files. Later on, additional processes will organize the images into directories specific to the bands necessary to make Melt Pond classification. 

In [ ]:
import zipfile

S2_path = 'ENTER THE FULL DIRECTORY PATH OF YOUR ZIPPED FILES'
for a in os.listdir(S2_path)[3:]:
    # The first portion of this loop looks for files which have been unzipped. Fully unpacked files are written to 
    # a new directory that ends with '.SAFE'. 
    if a.endswith('.zip'):
        a2= a[:-3]+'SAFE'
        if os.path.exists(S2_path+a2):
            pass
        else:
            with zipfile.ZipFile(S2_path+a, 'r') as zip_ref:
                zip_ref.extractall(S2_path)

# Managing Bands/Files

This portion of the script manages the specific files we need to run the classification as well as the additional True Color Image (TCI) complex. The classification requires bands 2, 3, 4, and 8 (Red, Blue Green, and NIR). The TCI is included for validation purposes and utility scripts are provided to make appropriate mosaics and georefferencing (check GitHub assets).

First, we check to see if band directories have been created, otherwise the program will create the directories necessary for images to be sent to appropriate locations.

In [ ]:
band_list = ['B02', 'B03', 'B04', 'B08', 'TCI']

for a in band_list:
    BandFolder = os.path.join(S2_path, a)
    #Skips directory creation if directories already exist
    if os.path.exists(BandFolder):
        pass
    #Creates directories if they don't already exist
    else:
        print('Directory does not exist, "' + a + '" will be created')
        BandDirec = os.path.join(S2_path, BandFolder)
        os.makedirs(BandDirec)

In this portion, *'a'* is the name of the folders inside the *'S2 Path'*. The individual bands are housed in directories within directories. The program moves the necessary .jp2 band files into seperate directories. Users may wish to discard the remaining *'.SAFE'* files but are encouraged to make back-ups in case they need to use other bands.

In [ ]:
for a in [a for a in os.listdir(S2_path) if a.endswith('.SAFE')]:
    fol=os.listdir(S2_path+a+'/GRANULE/')[-1]
    for d in ['B02','B03','B04','B08', 'TCI']:
        if len([b for b in os.listdir(S2_path+a+'/GRANULE/'+fol+'/IMG_DATA/') if b.endswith(d+'.jp2')])>0:
            c=[b for b in os.listdir(S2_path+a+'/GRANULE/'+fol+'/IMG_DATA/') if b.endswith(d+'.jp2')][0]
            if (os.path.exists(S2_path+d+'/'+c)): 
                pass
            else:
                shutil.move(S2_path+a+'/GRANULE/'+fol+'/IMG_DATA/'+c,S2_path+d+'/')
        